In [15]:
!pip install symspellpy rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.3 MB/s eta 0:00:0000:0100:01


In [19]:
import pandas as pd
import re
from symspellpy import SymSpell, Verbosity
from tqdm import tqdm
from rapidfuzz.distance import DamerauLevenshtein

In [4]:
train = pd.read_csv('/kaggle/input/spell-check-for-iron/train_OCR_error_iron.csv')
test = pd.read_csv('/kaggle/input/spell-check-for-iron/test_open_OCR_error_iron.csv')

In [5]:
def tokenize(sentence):
    return re.findall(r'\w+', str(sentence), re.UNICODE)

In [6]:
# Собираем словарь с частотами из train['Corrected_text']
freq_dict = {}
for text in tqdm(train['Corrected_text'], desc="Build freq dict"):
    words = tokenize(text)
    for w in words:
        freq_dict[w] = freq_dict.get(w, 0) + 1

Build freq dict: 100%|██████████| 47684/47684 [00:00<00:00, 128857.95it/s]


In [7]:
# Запишем словарь в файл для symspell
with open('custom_freq_dict.txt', 'w', encoding='utf-8') as f:
    for word, freq in freq_dict.items():
        f.write(f"{word} {freq}\n")

In [8]:
max_edit_distance = 2
sym_spell = SymSpell(max_dictionary_edit_distance=max_edit_distance, prefix_length=7)

sym_spell.load_dictionary('custom_freq_dict.txt', term_index=0, count_index=1)

True

In [9]:
def correct_word(word):
    if word in freq_dict or re.match(r'\s+|[^\w]', word):
        return word
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance)
    if suggestions:
        return suggestions[0].term
    else:
        return word

In [10]:
def correct_sentence(sentence):
    tokens = re.findall(r'\w+|\s+|[^\w\s]', str(sentence), re.UNICODE)
    return ''.join([correct_word(tok) for tok in tokens])

In [11]:
tqdm.pandas()
test['Corrected_text'] = test['OCR_with_errors'].progress_apply(correct_sentence)

100%|██████████| 11921/11921 [00:01<00:00, 7654.71it/s]


In [20]:
if 'Corrected_text' in train.columns:
    test_eval = test.merge(train[['OCR_with_errors', 'Corrected_text']], on='OCR_with_errors', suffixes=('_pred', '_true'))
    distances = [DamerauLevenshtein.distance(p, t) for p, t in zip(test_eval['Corrected_text_pred'], test_eval['Corrected_text_true'])]
    avg_distance = sum(distances) / len(distances) if distances else None
    if avg_distance is not None:
        print(f"Среднее редакционное расстояние (Damerau-Levenshtein): {avg_distance:.2f}")

Среднее редакционное расстояние (Damerau-Levenshtein): 0.76


In [22]:
import random

test['errors_found'] = test['OCR_with_errors'] != test['Corrected_text']
errors_df = test[test['errors_found']].copy()

if 'Corrected_text' in train.columns:
    errors_df = errors_df.merge(train[['OCR_with_errors', 'Corrected_text']], on='OCR_with_errors', suffixes=('_pred', '_true'))

sample_errors = errors_df.sample(10, random_state=42)  # случайные 10 ошибок

for idx, row in sample_errors.iterrows():
    print("OCR с ошибками:", row['OCR_with_errors'])
    print("Исправлено моделью:", row['Corrected_text_pred'] if 'Corrected_text_pred' in row else row['Corrected_text'])
    if 'Corrected_text_true' in row:
        print("Настоящий текст:", row['Corrected_text_true'])
    print("-" * 50)


OCR с ошибками: Багаев Н.
Исправлено моделью: Багӕв Н.
Настоящий текст: Багӕв Н.
--------------------------------------------------
OCR с ошибками: Джикаев, Ш.
Исправлено моделью: Джикӕв, Ш.
Настоящий текст: Джикӕв, Ш.
--------------------------------------------------
OCR с ошибками: Хетаегкаты Къостайы цард аемае архайд.
Исправлено моделью: Хетсегкаты Къостайы цард амал архайд.
Настоящий текст: Хетӕгкаты Къостайы цард ӕмӕ архайд.
--------------------------------------------------
OCR с ошибками: Багаев Н.
Исправлено моделью: Багӕв Н.
Настоящий текст: Багӕв Н.
--------------------------------------------------
OCR с ошибками: — Куыд ӕнхъӕлдтай, — фаерсы, — уыдзынӕ рохы?
Исправлено моделью: — Куыд ӕнхъӕлдтай, — фарсы, — уыдзынӕ рохы?
Настоящий текст: — Куыд ӕнхъӕлдтай, — фӕрсы, — уыдзынӕ рохы?
--------------------------------------------------
OCR с ошибками: Дым опускается черной тучей низко, Глухо стонут пылающие равниньь Тяжелым снопом падает человек, Словно Прометей, ни звука не из

In [23]:
submission = test[['id', 'Corrected_text']]
submission.to_csv('submission_symspell.csv', index=False)
print("Файл submission_symspell.csv успешно создан.")

Файл submission_symspell.csv успешно создан.
